#  Data analysis for Lexical Richness

In [1]:
#importing libraries
import os
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd

# Functions

In [26]:
#binary search for strings
def binarySearchArr(arr, x):
    l = 0
    r = len(arr)
    while (l <= r):
        m = l + ((r - l) // 2)
 
        res = (x == arr[m])
        # Check if x is present at mid
        if (res == True):
            return m
 
        # If x greater, ignore left half
    #problem is here
        elif (res == False):
            if (x>arr[m]):
                l = m+1
            elif (x<arr[m]):
                r=m-1
        # If x is smaller, ignore right half
    return -1

In [27]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [28]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

In [29]:
# reference code from previous research, tokinizes essays
# NEEDS UPDATING TO FIT CSV FILES

#function that does this
def calculate_tokens(file):
    essay = ''

    with open(file, 'r') as f: #used bc it's a file
        essay = f.read().lower()
        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


# Adding DFs

In [8]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)

In [3]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [10]:
loose.head()
#use text3 for word count?
#use text3_len for number of tokes, but doesn't actually get the tokens
#tokenize text3

,3Langs,Item,answer_id,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text1_len,text2_len,text3_len,text1,"text2 (line breaks/extra spaces removed, spaces added to reach 60)",text3 (edits made to fix word counts),Judgement,Notes
0,Y,C12,141.0,aj8,Chinese,Male,118.0,5.0,w,17.0,1.0,121.0,114.0,114.0,"\n\n\n Today, I am going to describe one of my...","Today, I am going to describe one of my classm...","Today, I am going to describe one of my classm...",1.0,NaN
1,Y,C13,143.0,az8,Chinese,Female,118.0,5.0,w,17.0,1.0,96.0,95.0,95.0,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,1.0,NaN
2,Y,K9,133.0,az2,Korean,Male,118.0,5.0,w,17.0,1.0,130.0,128.0,128.0,"When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...",1.0,NaN
3,Y,K11,135.0,at8,Korean,Female,118.0,5.0,w,17.0,3.0,104.0,105.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,1.0,NaN
4,Y,K20,188.0,eh9,Korean,Male,118.0,5.0,w,17.0,1.0,98.0,97.0,97.0,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,1.0,NaN


In [11]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [12]:
# Merging the datasets

merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs=student_ans.merge(course_, on='course_id') #merges the student-answers df with course df

In [13]:
stu_ans_crs

,anon_id,gender,native_language,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,do6,Female,Russian,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Female,Russian,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Female,Russian,31.0,1957,189,117,1,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Female,Russian,31.0,2164,190,117,1,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Male,Arabic,21.0,151,4,117,1,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...
46225,cy7,Female,Korean,NaN,47682,6066,1034,1,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Female,Turkish,NaN,47823,6066,1034,1,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Male,Chinese,NaN,47824,6066,1034,1,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Male,Chinese,NaN,47787,6066,1034,1,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


# Getting Ids of participants in all three of Whole Dataset

In [65]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1

In [66]:
sac_lvls_rkc

,anon_id,native_language,level_id
13,ax4,Chinese,5
14,ax4,Chinese,5
15,ax4,Chinese,5
16,ax4,Chinese,5
17,ax4,Chinese,5
...,...,...,...
46218,cz3,Korean,3
46219,cz3,Korean,3
46225,cy7,Korean,3
46227,fq6,Chinese,3


In [69]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('level_id','nunique'), lvl_nums=('level_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [70]:
lvl_list

,anon_id,n_uniq,lvl_nums
0,aa0,1,[5]
1,aa1,2,"[4, 3]"
2,aa3,1,[4]
3,aa5,1,[4]
4,aa8,2,"[5, 4]"
...,...,...,...
486,ha0,1,[5]
487,ha2,2,"[5, 4]"
488,ha6,2,"[3, 4]"
489,hb4,3,"[5, 4, 3]"


In [107]:
# will create a list of only the ids that have levels 3, 4, and 5 in their lvl_nums column
all_three=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        all_three.append(append)
    ind+=1

all_three.sort()
len(all_three)

47

In [108]:
all_anon=[]
for i in sac_lvls_rkc.anon_id:
    all_anon.append(i)
len(all_anon)
all_anon.sort()

sorted_sac=sac_lvls_rkc.sort_values(by=['anon_id']).reset_index(drop=True)

In [109]:
lang_dict_all={}
for i in all_three:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_all:
        lang_dict_all[key].append(val)
    else:
        lang_dict_all[key]=[val]
    
print(lang_dict_all)

{'Korean': ['ag9', 'an5', 'as0', 'ay1', 'be7', 'bv9', 'bz5', 'cc4', 'cj8', 'co5', 'cv3', 'cw0', 'dp5', 'ea4', 'eq8', 'es9', 'fj7', 'fp5', 'fs0', 'fu6', 'gc2', 'gg6', 'gq8', 'gz2', 'hb4'], 'Chinese': ['aq1', 'ar8', 'bf7', 'bl4', 'bl7', 'bp4', 'bz2', 'cb3', 'cf9', 'cz4', 'dk6', 'eq4', 'ev9', 'fi1', 'fj4', 'fw7', 'gb4', 'gx5'], 'Spanish': ['bj2', 'cm9', 'fa2', 'fy1']}


In [110]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls34=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls34.append(append)
    ind+=1

lvls34.sort()
len(lvls34)

121

In [111]:
print(lvl_list)

    anon_id  n_uniq   lvl_nums
0       aa0       1        [5]
1       aa1       2     [4, 3]
2       aa3       1        [4]
3       aa5       1        [4]
4       aa8       2     [5, 4]
..      ...     ...        ...
486     ha0       1        [5]
487     ha2       2     [5, 4]
488     ha6       2     [3, 4]
489     hb4       3  [5, 4, 3]
490     hc1       1        [5]

[491 rows x 3 columns]


In [112]:
lang_dict_34={}
for i in lvls34:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_34:
        lang_dict_34[key].append(val)
    else:
        lang_dict_34[key]=[val]
    
print(lang_dict_34)

{'Chinese': ['aa1', 'ag6', 'ai4', 'ap5', 'ap8', 'aq1', 'ar8', 'ax7', 'ba3', 'bf7', 'bl4', 'bl7', 'bp4', 'bu9', 'bz2', 'cb3', 'cf9', 'cl2', 'cl6', 'cq2', 'cs5', 'cz4', 'dg0', 'dk6', 'dl8', 'dm4', 'do7', 'dp1', 'dt4', 'dx1', 'eb6', 'ec5', 'eq4', 'ev5', 'ev9', 'ey8', 'ff6', 'fi1', 'fj4', 'fm2', 'fn8', 'fw7', 'fy3', 'gb4', 'gf2', 'gw5', 'gw8', 'gx5', 'gy3'], 'Korean': ['aa9', 'af1', 'ag9', 'ah9', 'an4', 'an5', 'as0', 'as4', 'ay1', 'bc0', 'be2', 'be7', 'bf1', 'bm0', 'br5', 'bv9', 'bz5', 'ca6', 'cc4', 'cf6', 'ch9', 'cj8', 'cn3', 'co5', 'cv3', 'cw0', 'cw6', 'cy5', 'dd1', 'dj6', 'dp5', 'ea4', 'ef4', 'eo2', 'eq8', 'es9', 'et1', 'ex0', 'fc9', 'fh7', 'fi4', 'fj7', 'fk1', 'fl4', 'fl5', 'fo3', 'fp5', 'fp9', 'fs0', 'fu6', 'fv7', 'fv9', 'gb7', 'gc2', 'gd1', 'gg6', 'gl4', 'gq8', 'gv1', 'gz2', 'ha6', 'hb4'], 'Spanish': ['bj2', 'ch0', 'cm9', 'cy6', 'en3', 'fa2', 'fe7', 'fg7', 'fy1', 'fy6']}


In [113]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls45=[]
ind=0
for i in lvl_list.anon_id:
    if ((4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls45.append(append)
    ind+=1

lvls45.sort()
len(lvls45)

160

In [114]:
lang_dict_45={}
for i in lvls45:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_45:
        lang_dict_45[key].append(val)
    else:
        lang_dict_45[key]=[val]
    
print(lang_dict_45)

{'Korean': ['aa8', 'ac5', 'ad1', 'ag9', 'al5', 'an5', 'ap4', 'aq5', 'as0', 'as7', 'au5', 'au6', 'aw6', 'ay1', 'be7', 'bq0', 'bu4', 'bv9', 'bw3', 'bz5', 'cc4', 'ce3', 'ch2', 'ci0', 'cj8', 'co5', 'cp0', 'cv3', 'cw0', 'cw1', 'dd9', 'df3', 'dh0', 'dm3', 'dp5', 'dy7', 'ea4', 'eb3', 'eb9', 'ec1', 'eg5', 'eq8', 'es0', 'es9', 'et3', 'ex3', 'fa0', 'fa5', 'fb4', 'fd6', 'ff1', 'fi5', 'fj7', 'fl0', 'fp5', 'fs0', 'ft2', 'fu6', 'fx4', 'fz8', 'ga1', 'gc2', 'ge6', 'gg2', 'gg6', 'gj0', 'gk5', 'gl1', 'gn0', 'gq8', 'gs3', 'gu0', 'gz2', 'ha2', 'hb4'], 'Chinese': ['ad7', 'ae9', 'af4', 'am5', 'an7', 'aq1', 'ar8', 'ar9', 'az8', 'bd7', 'bf2', 'bf7', 'bf9', 'bl4', 'bl7', 'bp2', 'bp4', 'br9', 'bv8', 'bw9', 'by5', 'by6', 'bz1', 'bz2', 'ca4', 'cb3', 'cd6', 'cf9', 'ci2', 'cj5', 'cl3', 'cs4', 'cz2', 'cz4', 'dc0', 'di7', 'dk6', 'dk9', 'dm8', 'dq9', 'du9', 'dw2', 'ei2', 'ei8', 'eo1', 'eq4', 'es4', 'et4', 'ev6', 'ev9', 'ff2', 'fg8', 'fi1', 'fj4', 'fk8', 'fo4', 'fq5', 'fw7', 'ga3', 'gb4', 'gf5', 'gl2', 'gm1', 'gn5', 'g

# Getting distributions of Loose Dataset

In [115]:
# dropping the columns that aren't needed for what we're doing right now
dropped_loose=loose.drop(columns=['3Langs','Item','answer_id','gender','course_id','class_id','question_id','version','text1_len',
                                     'text2_len','text3_len','text1','text2 (line breaks/extra spaces removed, spaces added to reach 60)',
                                     'text3 (edits made to fix word counts)','Judgement','Notes'])
final_loose=dropped_loose.dropna()


In [116]:
# this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=final_loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose


,anon_id,n_anon_ids_loose,lvl_nums_loose
0,aa0,1,[5.0]
1,aa3,1,[4.0]
2,aa8,2,"[4.0, 5.0]"
3,aa9,2,"[3.0, 4.0]"
4,ab6,1,[4.0]
...,...,...,...
278,gz2,2,"[4.0, 5.0]"
279,gz5,1,[4.0]
280,gz7,1,[4.0]
281,ha0,1,[5.0]


In [117]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

len(all_three_loose)


8

In [118]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
print(in34_loose)
print(len(in34_loose))

['aa9', 'ah9', 'an5', 'ap5', 'ar8', 'ax7', 'be2', 'bf1', 'bl4', 'bp4', 'bv9', 'bz2', 'cf6', 'ch0', 'cm9', 'cs5', 'cw6', 'cy5', 'ex0', 'ey8', 'fa2', 'fc9', 'fe7', 'fn8', 'fv7', 'fy1', 'gd1', 'gv1']
28


In [119]:
# creates list of participants in limited sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
print(in45_loose)
print(len(in45_loose))

['aa8', 'ac5', 'al5', 'an5', 'aq1', 'ar8', 'ar9', 'au5', 'au6', 'az8', 'bf2', 'bi4', 'bj2', 'bl4', 'bl7', 'bp4', 'bu4', 'bv9', 'bz2', 'cb3', 'cc4', 'ce3', 'cj5', 'cj8', 'co5', 'cv7', 'cw1', 'cz4', 'dc0', 'dc6', 'dd9', 'df4', 'dk0', 'dm8', 'dq9', 'dy7', 'ei8', 'eo8', 'et4', 'ev6', 'ev9', 'fa0', 'fa2', 'ff1', 'ff2', 'fg8', 'fj7', 'fk8', 'fp5', 'fu6', 'fx4', 'fx7', 'fy1', 'gb4', 'gf5', 'gm1', 'gq4', 'gq8', 'gu1', 'gz2', 'ha2']
61


# Answer counts for whole dataset

In [ ]:
e_counts=sorted_sac
e_counts["Number of Questions"]=e_counts.anon_id.count()
entire_counts=e_counts.groupby(['anon_id','native_language'])['Number of Questions'].count()
qcount_entire=entire_counts.reset_index()

In [102]:
qcount_entire

,anon_id,native_language,Number of Questions
0,aa0,Spanish,37
1,aa1,Chinese,49
2,aa3,Chinese,19
3,aa5,Chinese,5
4,aa8,Korean,100
...,...,...,...
486,ha0,Korean,4
487,ha2,Korean,59
488,ha6,Korean,123
489,hb4,Korean,97


In [120]:
espanish=0
echinese=0
ekorean=0
index=0
for i in qcount_entire["Number of Questions"]:
    anon_id=str(qcount_entire.at[index,'anon_id'])
    if anon_id in all_three:
        if qcount_entire.at[index,'native_language']=='Spanish':            
            espanish+=i
        elif qcount_entire.at[index,'native_language']=='Chinese':
            echinese+=i
        elif qcount_entire.at[index,'native_language']=='Korean':
            ekorean+=i
    index+=1
print('Counts in all 3\nSpanish counts: ',espanish,'\nChinese counts: ',echinese,'\nKorean counts: ',ekorean)

Counts in all 3
Spanish counts:  290 
Chinese counts:  1180 
Korean counts:  1925


In [124]:
espanish34=0
echinese34=0
ekorean34=0
index=0
for i in qcount_entire["Number of Questions"]:
    anon_id=str(qcount_entire.at[index,'anon_id'])
    if anon_id in lvls34:
        if qcount_entire.at[index,'native_language']=='Spanish':            
            espanish34+=i
        elif qcount_entire.at[index,'native_language']=='Chinese':
            echinese34+=i
        elif qcount_entire.at[index,'native_language']=='Korean':
            ekorean34+=i
    index+=1
print('Counts in 3 and 4\nSpanish counts: ',espanish34,'\nChinese counts: ',echinese34,'\nKorean counts: ',ekorean34)

Counts in 3 and 4
Spanish counts:  462 
Chinese counts:  2504 
Korean counts:  3841


In [125]:
espanish45=0
echinese45=0
ekorean45=0
index=0
for i in qcount_entire["Number of Questions"]:
    anon_id=str(qcount_entire.at[index,'anon_id'])
    if anon_id in lvls45:
        if qcount_entire.at[index,'native_language']=='Spanish':            
            espanish45+=i
        elif qcount_entire.at[index,'native_language']=='Chinese':
            echinese45+=i
        elif qcount_entire.at[index,'native_language']=='Korean':
            ekorean45+=i
    index+=1
print('Counts in 4 and 5\nSpanish counts: ',espanish45,'\nChinese counts: ',echinese45,'\nKorean counts: ',ekorean45)

Counts in 4 and 5
Spanish counts:  781 
Chinese counts:  4424 
Korean counts:  4916


# Tokinizing Essays for Loose

In [ ]:
# reference code from previous research

#function that does this
def calculate_tokens(file):
    essay = ''

    with open(file, 'r') as f:
        essay = f.read().lower()
        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


# don't show this code to anyone else, i'm a bad boy
directories = ['limited', 'loose', 'strictest']
tokenize_results = { 'filename': [], 'total_tokens': [], 'unique_tokens': [] }

for directory in directories:
    for file in os.listdir(f'./data/in/{directory}'):
        results = calculate_tokens(f'./data/in/{directory}/{file}')
        tokenize_results['filename'].append(file)
        tokenize_results['total_tokens'].append(results['total'])
        tokenize_results['unique_tokens'].append(results['unique'])

pd.DataFrame(data=tokenize_results)